# Test 03 — Line sensors (check each sensor one-by-one)

Goal: verify each of the 4 sensors can go True when placed over black tape.

You will be prompted:
- "Move sensor 1 onto tape"
- waits until that sensor reads True consistently
- then moves to sensor 2, 3, 4.


In [ ]:
from pathlib import Path
import sys

def add_repo_to_path():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            repo_root = p
            common_lib = p / 'common' / 'lib'
            if str(common_lib) not in sys.path:
                sys.path.insert(0, str(common_lib))
            print('Repo root:', repo_root)
            print('Added common/lib:', common_lib)
            return
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_to_path()

In [ ]:
import time
from line_sensors import LineSensors

ls = LineSensors(address=0x77, register=0x01)
print('LineSensors ready @ addr', hex(ls.address))

def bar(vals):
    return ''.join('■' if v else '·' for v in vals)

def wait_for_sensor(idx: int, stable_needed: int = 6, timeout_s: float = 40.0):
    print(f"\n--- Sensor {idx+1} ---")
    print('Move ONLY that sensor over black tape until it triggers...')
    stable = 0
    end = time.time() + timeout_s
    while time.time() < end:
        try:
            vals = ls.read()
        except OSError as e:
            print('I2C glitch:', e)
            time.sleep(0.05)
            continue
        hit = bool(vals[idx])
        stable = stable + 1 if hit else 0
        print(bar(vals), vals, 'hit' if hit else '')
        if stable >= stable_needed:
            print(f"✅ Sensor {idx+1} stable TRUE")
            return True
        time.sleep(0.12)
    print(f"❌ Timeout waiting for sensor {idx+1}")
    return False

ok = True
for i in range(4):
    ok = wait_for_sensor(i) and ok

print('\nRESULT:', 'PASS ✅' if ok else 'FAIL ❌')
